In [1]:
import pandas as pd
import os
import re
from sklearn.model_selection import train_test_split

### Identifying location of images in the directory structure for editing

In [6]:
#import the metadata for La Art after moving into the new directory
la_image_metadata = pd.read_csv('../data_samples/LaArt/latinamerican_art_with_filepaths.csv')

In [7]:
shape_initial = la_image_metadata.shape
print(shape_initial)

(628, 40)


### This is where 'file_name' column is used to refer to old naming pattern (outdated) TO-DO: Take any necessary following code (in this NB or download_LaArt) to download as new U.I image_fp with new naming pattern, instead of renaming/deduplicating the old.

In [8]:
#available_data = la_image_metadata.where(la_image_metadata.image_fp != 'Not Found').dropna(how='all')

In [9]:
#available_data['old_file_name'] = available_data.file_name.copy()

In [10]:
#selecting columns with title, forwarddisplayname, and objectid present
la_image_metadata = la_image_metadata.where(la_image_metadata.title.apply(pd.notna)).dropna(how='all')
la_image_metadata = la_image_metadata.where(la_image_metadata.forwarddisplayname.apply(pd.notna)).dropna(how='all')
la_image_metadata = la_image_metadata.where(la_image_metadata.objectid.apply(pd.notna)).dropna(how='all')

In [11]:
la_image_metadata['file_name'] = la_image_metadata['title'].apply(lambda x: x.replace(' ','_').replace('/', '&')) + '_' + la_image_metadata['forwarddisplayname'].apply(lambda x: x.replace(' ', '_').replace('/', '&')) + '_' + la_image_metadata['objectid'].apply(lambda x: str(int(x)) + '.jpg')

In [12]:
# image_fp column needed for downloadLa script
#this code sets the directory structure for top level
#can introduce the test/train subfolders as well in a new column
#then, the files can be downloaded and placed into this section
la_image_directory = '../latinamerican-2-imagefolder-split/'
la_image_metadata['directory'] = [la_image_directory] * len(la_image_metadata)

In [13]:
#subfolder split (train 70% /test 30%) need to be identified randomly
train_data, test_data = train_test_split(la_image_metadata, test_size=0.3)  

In [14]:
train_data['subfolder'] = ['train'] * len(train_data)
test_data['subfolder'] = ['test'] * len(test_data)

In [15]:
la_image_metadata = pd.concat([train_data, test_data]).reset_index(drop=True)

In [16]:
# adding expected filepath (directory + filename)
la_image_metadata['image_fp'] = la_image_metadata.directory + la_image_metadata.subfolder + '/' + la_image_metadata.file_name

In [17]:
# saves the selected parts to a new csv file to run the download script portion of downloadLa
la_image_fpaths = la_image_metadata.loc[:, ['expanded_url', 'file_name', 'image_fp']]

la_image_fpaths.to_csv('la_image_fpaths.csv', index=False)

KeyError: "['expanded_url'] not in index"

In [ ]:
shape_change_1 = la_image_metadata.shape
print('Shape starting: ', shape_initial)
print('Shape after edit 1: ', shape_change_1)

Shape starting:  (626, 44)
Shape after edit 1:  (626, 44)


In [18]:
#length of filenames must be below 256 to be used/stored by OS
fp_len = filepaths.apply(len)
fp_len.describe()

NameError: name 'filepaths' is not defined

In [19]:
filepaths[0][2:]

NameError: name 'filepaths' is not defined

### After this point any code is deprecated or a WIP

In [20]:
#using a file path searching algorithm to add the subdirectories the files exist in to make the full filepath
#this function requires the name to be formatted with extension
def find_files(filename, search_path):
    result = []

    # Walking top-down from the root
    for root, dir, files in os.walk(search_path):
        if filename in files:
            result.append(os.path.join(root, filename))
    if len(result) == 1:
        return result[0]
    else:
        return 'Not Found'

This needs to be ran after the files are downloaded

In [32]:
la_image_metadata['image_fp'] = la_image_metadata.file_name.apply(lambda x: find_files(x, la_image_directory))

In [35]:
#la_image_metadata['subfolder'] = la_image_metadata.image_fp.apply(lambda x: re.search('train|test', x)[0])

TypeError: 'NoneType' object is not subscriptable

### Identifying the duplicates which causes a reduction in the num of images downloaded due to file name overwriting issue (duplicates have same name, author) & redownloading with appropriate reidentification

In [38]:
#redownloading files with redundant names
#these are files we need to reinspect due to their multiplicity
series_num_objs = la_image_metadata.loc[:, ['title', 'forwarddisplayname']].groupby(
    ['title', 'forwarddisplayname']).apply(len)
series = series_num_objs[series_num_objs > 2]
series_groups = series.index
series_groups[0]
duplicates = None
for title, author in series_groups:
    dupe_partition = la_image_metadata.where(
        (la_image_metadata.title == title) & (la_image_metadata.forwarddisplayname == author)).dropna(how='all')
    if duplicates is None:
        duplicates = dupe_partition
    else:
        duplicates = pd.concat([duplicates, dupe_partition])

In [39]:
duplicates = duplicates.reset_index(drop=False)

In [46]:
duplicates.columns

Index(['index', 'iiifurl', 'iiifthumburl', 'title', 'displayDate_created',
       'roletype', 'role', 'forwarddisplayname', 'birthyear', 'deathyear',
       'ulanid', 'artistofngaobject', 'nationality', 'constituenttype',
       'demonym', 'pct_country_NGA', 'artists_countryCode',
       'artists_continent', 'artists_country', 'beginyear_artistAssigned',
       'endyear_artistAssigned', 'displaydate_artistAssigned',
       'country_artistAssigned', 'zipcode_artistAssigned', 'medium',
       'dimensions', 'inscription', 'markings', 'attribution',
       'visualBrowserClassification', 'parentID', 'isVirtual', 'portfolio',
       'series', 'volume', 'watermarks', 'uuid', 'viewtype', 'sequence',
       'width', 'height', 'maxpixels', 'assistivetext', 'depictstmsobjectid',
       'objectid', 'constituentid', 'expanded_url', 'directory', 'file_name',
       'image_fp'],
      dtype='object')

In [51]:
# Here we can see that the parentID can be an efficient way of grouping the duplicated titles (objectid being the childid)
duplicates.loc[:20,['title', 'forwarddisplayname', 'sequence', 'volume','portfolio', 'series', 'viewtype', 'parentID', 'objectid']]

,title,forwarddisplayname,sequence,volume,portfolio,series,viewtype,parentID,objectid
0,A Brazilian in Florida,Abraao Batista,0.0,NaN,NaN,NaN,primary,117249.0,126506.0
1,A Brazilian in Florida,Abraao Batista,0.0,NaN,NaN,NaN,primary,NaN,117249.0
2,A Brazilian in Florida,Abraao Batista,0.0,NaN,NaN,NaN,primary,117249.0,126504.0
3,A Brazilian in Florida,Abraao Batista,0.0,NaN,NaN,NaN,primary,117249.0,126507.0
4,A Brazilian in Florida,Abraao Batista,0.0,NaN,NaN,NaN,primary,117249.0,126501.0
5,A Brazilian in Florida,Abraao Batista,0.0,NaN,NaN,NaN,primary,117249.0,126502.0
6,A Brazilian in Florida,Abraao Batista,0.0,NaN,NaN,NaN,primary,117249.0,126500.0
7,A Brazilian in Florida,Abraao Batista,0.0,NaN,NaN,NaN,primary,117249.0,126505.0
8,A Brazilian in Florida,Abraao Batista,0.0,NaN,NaN,NaN,primary,117249.0,126503.0
9,Garden,Alejandro de Ávila,0.0,NaN,Garden,NaN,primary,139546.0,143119.0


In [52]:
#duplicates['old_fp'] = duplicates.directory + duplicates.subfolder + '/' + duplicates['title'].apply(lambda x: x.replace(' ','_').replace('/', '&')) + '_' + duplicates['lastname'].apply(lambda x: x.replace(' ', '_').replace('/', '&') + '.jpg')

In [53]:
#deleting old names (if not found thats a good thing)
#for name in duplicates.old_fp.unique():
#    try:
#        os.remove(name)
#    except FileNotFoundError:
#        print('Not found: ', name)

In [44]:
#import shutil
#import requests
#def download_image(url, folder, file_name, headers):
#    response = requests.get(url, headers=headers)
#    if response.status_code == 200:
#        with open(file_name, "wb") as f:
#            f.write(response.content)
#        shutil.move(file_name, folder + file_name)
#    else:
#        print(response.status_code)
##Define HTTP Headers
#headers = {
#    "User-Agent": "Chrome/51.0.2704.103",
#}
#import time
#for i in range(0, len(duplicates)):
#    #Define URL of an image
#    url = duplicates.expanded_url[i]
#    # Define image file name
#    file_name = duplicates.file_name[i]
#    # Define location to move file
#    folder = duplicates.directory[i] + duplicates.subfolder[i] + '/'
#    if len(file_name[:-4]) > 250:
#        num_remove = len(file_name[:-4]) - 250
#        file_name = file_name[:-(4 + num_remove)] + '.jpg'
#    # Download image
#    #timer delay (15 seconds)
#    time.sleep(15)
#    download_image(url, folder, file_name, headers)

In [54]:
#available_data

In [63]:
#available_data = available_data.reset_index(drop=False)

In [55]:
#for i in range(0, len(available_data)):
#    # Renaming filenames in overall metadata & in directory to recognize new naming pattern =, only if not already in to_download
#    folder = available_data.directory[i] + available_data.subfolder[i] + '/'
#    old_file_name = available_data.old_file_name[i]
#    new_file_name = available_data.file_name[i]
#    if len(new_file_name[:-4]) > 250:
#        num_remove = len(new_file_name[:-4]) - 250
#        new_file_name = new_file_name[:-(4 + num_remove)] + '.jpg'
#    #renaming the file in the directory
#    oldName = os.path.join(folder, old_file_name)
#    newName = os.path.join(folder, new_file_name)
#    if os.path.exists(oldName) and os.path.exists(newName):
#        os.remove(oldName)
#        available_data['file_name'][i] = new_file_name
#        print('Old Name Deleted: ', old_file_name)
#    elif os.path.exists(oldName) and not os.path.exists(newName):
#        # update to new naming convention and delete oldName and update metadata
#        os.rename(oldName, newName)
#        #renaming the file in the metadata
#        available_data['file_name'][i] = new_file_name
#        print('New Name: ', new_file_name)

The output of downloaded_LaArt should not be used as it is a error-correction test file and any changes to latinamerican_art.csv should be written to a new (name)\_pipe\_(part#).csv file of the same name. (e.g. downloadla_pipe_2.csv)

In [69]:
#available_data.to_csv('../data_samples/results/whole_set_results/downloaded_LaArt.csv', index=False)